In [1]:
def obtain_ids(items_ext):
    href = items_ext.getAttribute('href')
    filename = href.split('#')[0]
    id_tmp = href.split('#')[-1].split(')..id(')
    init_id = id_tmp[0].split('id(')[-1].split(')')[0]
    final_id = None
    if len(id_tmp) > 1:
        final_id = id_tmp[1].split(')')[0]
    return filename, init_id, final_id

In [2]:
def obtain_node_from_textname(path, id_arr, tag):
    mydoc = minidom.parse(path)
    items_dact = mydoc.getElementsByTagName(tag)
    nodes = []
    for item in items_dact:
        nite_id = item.getAttribute('nite:id')
        if nite_id in id_arr:
            nodes.append(item)
    return nodes

In [3]:
def obtain_dialogue_act_word_references(word_nodes):
    """ Obtain words from word references

        :param word_nodes: nodes representing words
        :return: string with concatenated words
    """
    words = ""

    for word_node in word_nodes:
        words += word_node.firstChild.data + " "

    return words

In [ ]:
from os import listdir
from tqdm import tqdm
from xml.dom import minidom
from os.path import isfile, join
mypath=r"C:\\Users\\mohan.adabala\\Music\\ML Pracice\\NLP\\Document Summarization\\AMICorpusXML-master\\data\\ami_public_manual_1.6.2\\dialogueActs\\"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
contains="adjacency-pairs"
onlyfiles_new=[files for files in onlyfiles if contains not in files]
Sent_all=[]
Label_all=[]
for fil in tqdm(onlyfiles_new):
#     print()
    dactpath=mypath+fil
    mydoc = minidom.parse(dactpath)
    items_dact = mydoc.getElementsByTagName('dact')
    nodes = []
    sentence_segment = []
    sum_dir = r"C:\\Users\\mohan.adabala\\Music\\ML Pracice\\NLP\\Document Summarization\\AMICorpusXML-master\\data\\ami_public_manual_1.6.2\\words\\"
# nite:pointer
    label=[]
# items_extsumm=[]
    for segment_node in items_dact:
            # Obtain child node with words information
        for seg in segment_node.childNodes:
            if seg.attributes is not None:
                if seg.nodeName == 'nite:pointer':
                    items_extsumm = seg
        filename, init_id, final_id = obtain_ids(items_extsumm)
        label.append(init_id)
    Label_all.append(label)        
# for it in items_dact:
    for segment_node in items_dact:
            # Obtain child node with words information
        for seg in segment_node.childNodes:
            if seg.attributes is not None:
                if seg.nodeName == 'nite:child':
                    items_extsumm = seg  # segment_node.childNodes[3]  # Element

        filename, init_id, final_id = obtain_ids(items_extsumm)
        id_arr = [init_id]
        if final_id is not None:
            init_id_index = int(init_id.split('words')[-1])
            final_id_index = int(final_id.split('words')[-1])
            root_name = final_id.split(str(final_id_index))[0]
            for i in range(init_id_index + 1, final_id_index + 1):
                id_arr.append(root_name + str(i))
        word_nodes = obtain_node_from_textname(sum_dir + filename, id_arr, tag='w')
        sentence_segment.append(obtain_dialogue_act_word_references(word_nodes))        
    Sent_all.append(sentence_segment)            

 94%|████████████████████████████████████████████████████████████████████████▉     | 520/556 [3:14:26<06:42, 11.18s/it]

In [ ]:
flat_sent = [item for sublist in Sent_all for item in sublist]
flat_label = [item for sublist in Label_all for item in sublist]

In [ ]:
flat_sent[100],flat_label[100]

In [ ]:
import pandas as pd

data = {'sentence':  flat_sent,
        'label': flat_label
        }
df = pd.DataFrame (data, columns = ['sentence','label'])
print (df)
# 5, 11
df.to_pickle("./sent_annot.pkl")

In [ ]:
df['label'].value_counts().sort_values(ascending=False)

In [ ]:
pd.set_option('display.max_colwidth', -1)
american = df['label'] == "ami_da_6"
df[american]